$$\require{mhchem}$$

# Photorespiration model


## Original

Publication: Yokota, A., Kitaoka, S., Miura, K. et al. Reactivity of glyoxylate with hydrogen perioxide and simulation of the glycolate pathway of C3 plants and Euglena . Planta 165, 59–67 (1985). https://doi.org/10.1007/BF00392212


### Scheme

<img src="schemes/model.svg">



### Reactions

| Enzyme                         | Reaction | Stoichiometries                                             |
| ------------------------------ | -------- | ----------------------------------------------------------- |
| (influx)                       | R0       | $$\ce{-> glycolate}$$                                       |
| glycolate oxidase              | R1       | $$\ce{glycolate + O2 -> glyoxylate + H2O2}$$                |
| glycine transaminase           | R2       | $$\ce{glyoxylate + glutamate -> glycine + ketoglutarate}$$  |
| glycine decarboxylase          | R3       | $$\ce{2 glycine + NAD -> serine + NADH + CO2 + NH4}$$       |
| serine glyoxylate transaminase | R4       | $$\ce{serine + glyoxylate -> hydroxypyruvate + glycine}$$   |
| glycerate dehydrogenase        | R5       | $$\ce{hydroxypyruvate + NADH -> glycerate + NAD}$$          |
| catalase                       | R6       | $$\ce{2 H2O2 -> 2 H2O + O2}$$                               |
| nonenzymatic                   | R7       | $$\ce{glyoxylate + H2O2 -> CO2 + formate}$$               |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from modelbase.ode import Model, Simulator

In [ ]:
def constant(k):
    return k


def mass_action_1(s1, k_fwd):
    return k_fwd * s1


def mass_action_2(s1, s2, k_fwd):
    return k_fwd * s1 * s2


def michaelis_menten(s, vmax, km):
    return vmax * s / (km + s)


def ping_pong_bi_bi(s1, s2, vmax, km_s1, km_s2):
    return (
        vmax * s1 * s2 / ((1 / (km_s1 * km_s2)) + (s1 / km_s1) + (s2 / km_s2) + s1 * s2)
    )

In [ ]:
cf = 0.145  # conversion factor 1mM = 0.145 micromoles per mg Chl

parameters = {
    "v_influx": 60.0 / cf,  # mM per hour
    "vmax_glycolate_oxidase": 100.0 / cf,  # mM per hour
    "vmax_glycine_transaminase": 143.0 / cf,  # mM per hour
    "vmax_glycine_decarboxylase": 100.0 / cf,  # mM per hour
    "vmax_ser_gly_transaminase": 159.0 / cf,  # mM per hour
    "vmax_glycerate_dehydrogenase": 398.0 / cf,  # mM per hour
    "vmax_catalase": 760500 / cf,  # mM per hour
    "km_glycolate_oxidase": 0.06,  # mM
    "km_glycine_transaminase": 3.0,  # mM
    "km_glycine_decarboxylase": 6.0,  # mM
    "km_transaminase_glyxoylate": 0.15,  # mM
    "km_transaminase_serine": 2.72,  # mM
    "km_glycerate_dehydrogenase": 0.12,  # mM
    "km_catalase": 137.9,  # mM
    "k_r7": 27.0,  # mM / hour
}

In [ ]:
compounds = [
    "glycolate",
    "glyoxylate",
    "glycine",
    "serine",
    "hydroxypyruvate",
    "H2O2",
]

In [ ]:
m = Model()
m.add_parameters(parameters)
m.add_compounds(compounds)
m.add_reaction(
    "R0",
    function=constant,
    stoichiometry={"glycolate": 1},
    parameters=["v_influx"],
)
m.add_reaction(
    "R1",
    function=michaelis_menten,
    stoichiometry={"glycolate": -1, "glyoxylate": 1, "H2O2": 1},  # o2 is ignored
    parameters=["vmax_glycolate_oxidase", "km_glycolate_oxidase"],
)
m.add_reaction(
    "R2",
    function=michaelis_menten,
    stoichiometry={
        "glyoxylate": -1,
        "glycine": 1,
    },  # glutamate & ketoglutarate are ignored
    parameters=["vmax_glycine_transaminase", "km_glycine_transaminase"],
)
m.add_reaction(
    "R3",
    function=michaelis_menten,
    stoichiometry={"glycine": -1, "serine": 0.5},  # NAD(H), co2 & nh4 are ignored
    parameters=["vmax_glycine_decarboxylase", "km_glycine_decarboxylase"],
)
m.add_reaction(
    "R4",
    function=ping_pong_bi_bi,
    stoichiometry={"glyoxylate": -1, "serine": -1, "glycine": 1, "hydroxypyruvate": 1},
    parameters=[
        "vmax_ser_gly_transaminase",
        "km_transaminase_glyxoylate",
        "km_transaminase_serine",
    ],
)
m.add_reaction(
    "R5",
    function=michaelis_menten,
    stoichiometry={"hydroxypyruvate": -1},  # NAD(H) & glycerate is ignored
    parameters=["vmax_glycerate_dehydrogenase", "km_glycerate_dehydrogenase"],
)
m.add_reaction(
    "R6",
    function=michaelis_menten,
    stoichiometry={"H2O2": -1},  # H2O & O2 is ignored
    parameters=["vmax_catalase", "km_catalase"],
)
m.add_reaction(
    "R7",
    function=mass_action_2,
    stoichiometry={"glyoxylate": -1, "H2O2": -1},  # co2 and formate are ignored
    parameters=["k_r7"],
)

In [ ]:
y0 = {
    "glycolate": 0,
    "glyoxylate": 0,
    "glycine": 0,
    "serine": 0,
    "hydroxypyruvate": 0,
    "H2O2": 0,
}

s = Simulator(m)
s.initialise(y0)
t, y = s.simulate(1)
fig, ax = s.plot_grid(
    [
        ["glycolate", "glyoxylate", "glycine"],
        ["serine", "hydroxypyruvate"],
        ["H2O2"],
    ],
    figure_title="Wildtype",
    xlabels="Time [h]",
    ylabels="Concentration [mM]",
)
fig.tight_layout()
plt.show()

fig, axs = plt.subplots(2, 2, figsize=(10, 6), sharex=True)
for ax, rate_name in zip(axs.flatten(), ["R0", "R3", "R6", "R7"]):
    s.plot_flux_selection([rate_name], ax=ax, xlabel="", ylabel="")
    ax.legend([])
    ax.set_title(rate_name)
for ax in axs[:, 0]:
    ax.set_ylabel("Fluxes [mM/h]")
for ax in axs[-1, :]:
    ax.set_xlabel("Time [h]")
fig.suptitle("Wildtype")
fig.tight_layout()
plt.show()

The plot above shows the following conclusions from the paper

- We conclude that H202 produced during glycolate oxidation in peroxisomes is decomposed by catalase (R6) but not by the reaction with glyoxylate (R7), and that photorespiratory CO2 originates from glycine (R3), but not from glyoxylate (R7), in C3 plants.
- In the simulation of the glycolate pathway of Euglena, the rate constant was high enough to ensure the decarboxylation of glyoxylate by H202 to produce photorespiratory CO2 during the glycolate metabolism of this organism

## Extension

- Implementation: Dipali Singh

### Scheme

<img src="schemes/model_extended.svg">

### Reactions

| Enzyme                         | Reaction | Stoichiometries                                             |
| ------------------------------ | -------- | ----------------------------------------------------------- |
| hydroxypyruvate decarboxylase  | R8       | $$\ce{hydroxypyruvate + H2O2 -> glycolate + CO2}$$          |


In [ ]:
m2 = m.copy()
m2.add_parameters(
    {
        "k_hydroxypyruvate_decarboxylase": 11.74,  # mM / hour
    }
)

m2.add_reaction(
    "R8",
    function=mass_action_2,
    stoichiometry={
        "hydroxypyruvate": -1,
        "H2O2": -1,
        "glycolate": 1,
    },  # co2 is ignored
    parameters=["k_hydroxypyruvate_decarboxylase"],
)

### Simulation

In [ ]:
with m2 as catalase_mutant:
    catalase_mutant.update_parameter(
        "vmax_catalase", 0.005 * m.parameters["vmax_catalase"]
    )

    s2 = Simulator(catalase_mutant)
    s2.initialise(y0)
    t, y = s2.simulate(1)
    fig, axs = s2.plot_grid(
        [
            ["glycolate", "glyoxylate", "glycine"],
            ["serine", "hydroxypyruvate"],
            ["H2O2"],
        ],
        figure_title="Catalase mutant",
        xlabels="Time [h]",
        ylabels="Concentration [mM]",
    )
    fig.tight_layout()
    plt.show()

    fig, axs = plt.subplots(2, 2, figsize=(10, 6), sharex=True)
    for ax, rate_name in zip(axs.flatten(), ["R3", "R6", "R7", "R8"]):
        s2.plot_flux_selection([rate_name], ax=ax, xlabel="", ylabel="")
        ax.legend([])
        ax.set_title(rate_name)
    for ax in axs[:, 0]:
        ax.set_ylabel("Fluxes [mM/h]")
    for ax in axs[-1, :]:
        ax.set_xlabel("Time [h]")
    fig.suptitle("Catalase mutant")
    fig.tight_layout()
    plt.show()

In the case of the catalase mutant, the non-enzymatic reaction of $\ce{H2O2}$ with glyoxylate (R7) becomes more important, while the hydroxypyruvate decarboxylase (R8) is barely active.